
# <center>  Random Forest </center>

## Descripción
En esta sección se analiza el algoritmo de Random Forest.

## Contenido
* Importación de librerias y módulos
* Carga dataset de trabajo
* Random Forest

## Requisitos previos

* Haber completado los cursos:
  - Introducción a Python
  - Estadística para Ciencia de Datos
  - Introducción a Machine Learning



In [0]:
import numpy as np
import pandas as pd

import sklearn
from sklearn import model_selection
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import load_breast_cancer

import matplotlib
import matplotlib.pyplot as plt

In [0]:
# Verificación de versiones de librerías usadas
print('La versión de Numpy es {}'.format(np.__version__))
print('La versión de Pandas es {}'.format(pd.__version__))
print('La versión de Scikit-learn es {}'.format(sklearn.__version__))
print('La versión de Matplotlib es {}'.format(matplotlib.__version__))

In [0]:
RANDOM_STATE = 42

## 1. Dataset de Trabajo

**``df``:** Contiene las features de una imagen digitalizada de un aspirado de aguja fina de una masa mamaria. Se describen las características de los núcleos celulares presentes en la imagen.

**Objetivo:** Clasificar si un tumor es benigno o malino a partir de los features correspondientes.

In [0]:
# Cargar el dataset
cancer = load_breast_cancer()
df = pd.DataFrame(cancer.data)
df.columns = cancer.feature_names

In [0]:
# Diccionario de datos
print(cancer.DESCR)

In [0]:
# Imprimir la información del DataFrame
df.info()

In [0]:
# Imprimir las 5 primeras filas 
df.head()

In [0]:
# Tamaño de la tabla de datos
df.shape

### 1.1. Análisis exploratorio

In [0]:
# Imprimir estadísticas descriptivas
df.describe(include = np.number)

In [0]:
# Identificación de valores faltantes
df.isna().sum()

**Delimitación de variable objetivo**

In [0]:
# Declarar variable objetivo
diagnostico = pd.Series(cancer.target, name = 'diagnostico')

# Imprimir conteo por categorías 
diagnostico.value_counts(normalize=False)

### 1.2. Preparación de los datos

In [0]:
X = df.loc[:, :]
y = diagnostico

print("Dimensiones de X", X.shape)
print("Dimensiones de y", y.shape)

## 2. Preparación de los datos
### 2.1 Train-test split

In [0]:
# Separar en datos de entrenamiento y test
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, 
                                                                    test_size=0.2, 
                                                                    random_state=RANDOM_STATE)

## 3 Random Forest

### 3.1 Etapas
Usando Scikit-learn, debemos primero crear una instancia de ```sklearn.ensemble.RandomForestClassifier```, e invocar los métodos necesarios:

1. Preparar los datos X, y
2. Crear instancia de ```sklearn.ensemble.RandomForestClassifier```:
```
classifier = RandomForestClassifier(random_state=RANDOM_STATE,
                                    n_estimators=1000,
                                    max_features=4,
                                    min_sample_leaf=10,
                                    random_state=0,
                                    n_jobs=2
                                   )
```
3. Entrenar:
```
classifier.fit(X, y)
```
4. Predecir:
```
y_hat = classifier.predict(X_test)
```

A continuación describiremos las opciones más comunes:


---
**```sklearn.ensemble.RandomForestClassifier(criterion=criterion, n_estimators=n_estimators, max_depth=max_depth, max_features=max_features, min_samples_leaf=min_samples_leaf, n_jobs=n_jobs, random_state=random_state)```**
Crea una instancia de un clasificador de Random Forest:

* ```criterion``` **string**. Tipo de criterio de pureza usado para realizar el split. Puede ser ```'gini'``` o ```'entropy'```.
* ```max_depth``` **int o None**. Especifica la máxima profundidad del árbol. Si es dejada en **None**, se crecerá el árbol hasta que todos los nodos tenga la máxima pureza o hasta que las hojas del árbol contengan menos de ```min_samples_split``` muestras.
* ```min_samples_split``` **int o float**. Es el mínimo número de muestras necesarias para hacer el split de un nodo interno. Por defecto es ```min_samples_split=2```.
* ```max_features``` **int, float, string o None**. Número de atributos a considerar cada vez que se hace un split. 
  - Cuando es **int**, considera este número en cada split.
  - Si es **float**, **int(max_features * n_features)** atributos son considerados.
  - Si es ```max_features='auto'```, ```max_features=sqrt(n_features)```.
  - Si es ```max_features='sqrt'```, ```max_features=sqrt(n_features)```.
  - Si es ```max_features='log2'```, ```max_features=log2(n_features)```.
  - Si es **None**, ```max_features=n_features```.
* ```min_samples_leaf``` **int o float**. Es el mínimo número de muestras requeridas para un nodo hoja.
* ```n_jobs``` **int o None**. El número de jobs a ejecutar en paralelo. Con **None** solo se usa 1.
* ```random_state``` **int**. Es el seed utilizado para garantizar reproducibilidad.

Documentación: https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html

In [0]:
# Creamos instancia de RandomForestClassifier
RFclassifier = RandomForestClassifier(random_state=RANDOM_STATE, 
                                      n_jobs=-1)
# Entrenamiento
RFclassifier.fit(X_train, y_train)

### 3.2. Cross-Validación

In [0]:
alphas  = [1, 0.1, 0.01, 0.001, 0.0001, 0]
max_features = ['auto', 'sqrt', 'log2']

In [0]:
# Empleamos validación cruzada a través de GridSearch
# Computacionalmente intensivo

grid_dtr = {'ccp_alpha': alphas,
            'max_features': max_features}

k = 10
grid = model_selection.GridSearchCV(cv=k, 
                                    param_grid=grid_dtr, 
                                    estimator=RFclassifier,
                                    scoring='roc_auc', 
                                    n_jobs= -1)
grid.fit(X_train, y_train)

In [0]:
grid.best_params_

### 3.3 Ajustamos el algoritmo con los parámetros optimizados

In [0]:
# Creamos instancia de RandomForestClassifier
RFclassifier_opt = RandomForestClassifier(n_estimators=10000,
                                          ccp_alpha=grid.best_params_['ccp_alpha'],
                                          max_features=grid.best_params_['max_features'],
                                          random_state=RANDOM_STATE, 
                                          n_jobs=-1)
# Entrenamiento
RFclassifier_opt.fit(X_train, y_train)

### 3.4 Predicción del modelo

In [0]:
# Calculamos las predicciones en datos de training
y_pred_train = RFclassifier_opt.predict(X_train)
# Calculamos las predicciones en datos de testing
y_pred_test  = RFclassifier_opt.predict(X_test)

In [0]:
print('Métricas en los datos de train')
print('Matriz de confusion')
print(metrics.confusion_matrix(y_train, y_pred_train))
print('Accuracy', metrics.accuracy_score(y_train, y_pred_train))


print('\n Métricas en los datos de test')
print('Matriz de confusion')
print(metrics.confusion_matrix(y_test, y_pred_test))
print('Accuracy', metrics.accuracy_score(y_test, y_pred_test))

#Ejercicio

1. Crear una nueva instancia del algoritmo de **Random Forest** incluyendo:
````
  * class_weight= "balanced"
  * max_depth=3
  * n_jobs=-1
  * random_state=RANDOM_STATE
````

2. Ejecutar el proceso de validación cruzada evaluando el siguiente grid de parámetros en los datos de entrenamiento y empleando como métrica ``scoring='roc_auc'``: 
````
  * criterion: ['giny', 'entropy']
````
  
3. Realizar las predicciones para los datos de test
4. Imprimir la matriz de confusión
5. Obtener la métrica del accuracy (exactitud) en los datos de test

In [0]:
#Su código aquí


In [0]:
# Declarar la instancia del algoritmo
rf_ejercicio = RandomForestClassifier(class_weight='balanced', 
                                      max_depth=3,
                                      random_state=RANDOM_STATE,
                                      n_jobs=-1)

# Emplear validación cruzada
grid_e = {'criterion': ['giny', 'entropy']}

k = 10
grid_rf = model_selection.GridSearchCV(cv=k, 
                                       param_grid=grid_e, 
                                       estimator=rf_ejercicio,
                                       scoring='roc_auc', 
                                       n_jobs= -1)
# Entrenar el algoritmo
grid_rf.fit(X_train, y_train)

# Realizar las predicciones
y_pred_testRF_e = grid_rf.predict(X_test)

# Imprimir métricas de evaluación
print('\n Métricas en los datos de test')
print('Matriz de confusion')
print(metrics.confusion_matrix(y_test, y_pred_testRF_e))
print('Accuracy', metrics.accuracy_score(y_test, y_pred_testRF_e))

Métricas en los datos de test
Matriz de confusion
[[41 2]
 [ 1 70]]
Accuracy 0.9736842105263158